# Simple RNN

In ths notebook, we're going to train a simple RNN to do **time-series prediction**. Given some set of input data, it should be able to generate a prediction for the next time step!
<img src='assets/time_prediction.png' width=40% />

> * First, we'll create our data
* Then, define an RNN in PyTorch
* Finally, we'll train our network and see how it performs

### Import Libraries and Tools

In [ ]:
import numpy as np
import torch
from torch import nn

# visualization and plotting
%matplotlib inline 
import matplotlib.pyplot as plt

### Generate dummy time-series data

In [5]:
plt.figure(figsize=(8,5))

# how many time steps/data pts are in one batch of data
seq_length = 20

# generate evenly spaced data pts
time_steps = np.linspace(0, np.pi, seq_length + 1)
data = np.sin(time_steps)
data.resize((seq_length + 1, 1)) # size becomes (seq_length+1, 1). each sequence only has 1 feature

x = data[:-1] # all but the last piece of data
y = data[1:] # all but the first

# display the data
plt.plot(time_steps[1:], x, 'r.', label='input, x') # x
plt.plot(time_steps[1:], y, 'b.', label='target, y') # y

plt.legend(loc='best')
plt.show()

## Define the RNN

Next, we define an RNN in PyTorch. We'll use `nn.RNN` to create an RNN layer, then we'll add a last, fully-connected layer to get the output size that we want. An RNN takes in a number of parameters:
* **input_size** - the number of expected features in the input x
* **hidden_dim** - the number of features in the RNN output and in the hidden state
* **n_layers** - the number of layers that make up the RNN, typically 1-3; greater than 1 means that you'll create a stacked RNN
* **batch_first** - If True, then the input and output tensors are provided as (batch, seq, feature). Default input and output are of shape (seq_len, batch, input_size).
* **dropout** - If non-zero and less than 1.0, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0
* **bidirectional** – If True, becomes a bidirectional LSTM. Default: False

The outputs of RNN are `output` features and `h_n` hidden state.
* **output**: tensor of shape `(seq_len, batch_size, num_dir * hidden_size)` containing the output features (h_t) from the last layer of the RNN, for each t. 
* **h_n**: tensor of shape `(num_layers * num_dir, batch, hidden_size)` containing the hidden state for t = seq_len.

Take a look at the [RNN documentation](https://pytorch.org/docs/stable/nn.html#rnn) to read more about recurrent layers.

In [ ]:
class SimpleRNN(nn.Module):
    """Simple RNN with a fully connected layer.
        
        Args:
            * input_size - the number of expected features in the input x
            * output_size - the number of expected feature in output x
            * hidden_dim - the number of features in the RNN output and in the hidden state
            * n_layers - the number of layers that make up the RNN, typically 1-3; greater than 1 means that you'll create a stacked RNN
    """
    def __init__(self, input_size, output_size, hidden_dims, n_layers):
        super(SimpleRNN, self).__init__()
        
        self.hidden_dims = hidden_dims # channel size of rnn output
        
        # initialise and define RNN configurations
        self.rnn = nn.RNN(input_size, hidden_dim, 
                          n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, h_n):
        """Feed-forward method.
        
            Args:
                * x - input tensor of shape (batch_size, seq_length, input_size)
                * h_n - tensor shape (num_layers, batch_size, hidden_size) containing hidden states
        """
        r_out, h_n = self.rnn(x, h_n)
        
        # reshape r_out to be (batch_size * seq_length, hidden_dim)
        r_out = r_out.view(-1, self.hidden_dim)
        
        output = self.fc(r_out)
        return output, h_n

In [ ]:
# test that dimensions are as expected
test_rnn = RNN(input_size=1, output_size=1, hidden_dim=10, n_layers=2)

# generate evenly spaced, test data pts
time_steps = np.linspace(0, np.pi, seq_length)
data = np.sin(time_steps)
data.resize((seq_length, 1))

test_input = torch.Tensor(data).unsqueeze(0) # give it a batch_size of 1 as first dimension
print('Input size: ', test_input.size())

# test out rnn sizes
test_out, test_h = test_rnn(test_input, None)
print('Output size: ', test_out.size())
print('Hidden state size: ', test_h.size())